In [176]:
import sys
sys.path.append("..")
import numpy as np 
import torch
from relie.utils.so3_tools import so3_hat, so3_vee, so3_exp, so3_log

In [79]:
def det_jac_so3(x):
    theta = x.norm(2,-1)
    return 2*(1 - torch.cos(theta))/(theta**2)

def so3_inv(el):
    return el.transpose(-2,-1)
    

In [158]:
dim = 3
eps = 0.005



def compute_approx_jacobian(points, vee, exp, log, inv, eps = 0.01):
    points = points.unsqueeze(-2)
    dim = points.shape[-1]
    basis = torch.eye(dim).unsqueeze(0)
    group_delta = exp(basis*eps + points)
    points_inv = inv(exp(points))
    normal_coord = vee(log(points_inv@group_delta))
    estimated_det_jac = np.linalg.det(normal_coord.numpy())/((eps)**dim)
    return torch.tensor(estimated_det_jac)

def approximate_so3_jacobian(points, eps = 0.01):
    return compute_approx_jacobian(points, so3_vee, so3_exp, so3_log, so3_inv, eps)

In [160]:
center = torch.tensor(np.random.normal(0,1,(10,3)),dtype = torch.float32)
estimated_det_jac = approximate_so3_jacobian(center, eps=eps)

In [161]:
print(estimated_det_jac - det_jac(center))

tensor(1.00000e-04 *
       [ 1.1545, -0.4900, -0.6735, -0.0721,  0.0232,  1.0705,  0.9453,
         0.4071,  0.0519,  0.5972])
